# Run OpenQASM Benzene simulation

In [ ]:
from qdk import init, TargetProfile
from qdk.openqasm import circuit, compile, run
from qdk.widgets import Circuit, Histogram

from pathlib import Path
src = Path("benzene_diradical_injected_rotation_measurement_circuit_0.qasm").read_text()

init(target_profile=TargetProfile.Base)
qir = compile(src)

## Show the circuit for the program to be simulated

In [ ]:
Circuit(circuit(src))

## Show noiseless results from the existing sparse simulator

In [ ]:
# Run using the default sparse simulator
sim_results = run(src, shots=1024)
Histogram(sim_results, labels="kets")

## GPU simulation with noise
__Note:__ Simulating the high-level operations. _Not_ compiled to any specific target hardware. This is directly using a simulator API we may choose not to expose initially.

In [ ]:
from qdk.simulation import NoiseConfig, run_qir

# Use the GPU simulator with noise
noise = NoiseConfig()
noise.cx.set_depolarizing(0.0005)
noise.h.set_depolarizing(0.0005)
noise.s_adj.set_depolarizing(0.0005)
noise.rz.set_depolarizing(0.0007)
noise.mresetz.set_depolarizing(0.0002)

output = run_qir(qir, shots=1024, noise=noise, seed=None, type="gpu")
Histogram(output, labels="kets")

## View how this would run on an NeutralAtomDevice-like device

In [ ]:
from qdk.simulation import NeutralAtomDevice, NoiseConfig

device = NeutralAtomDevice()
device.trace(qir)

## Simulate running on an NeutralAtomDevice with noise

In [ ]:
noise = NoiseConfig()
noise.cz.set_depolarizing(0.05)
noise.sx.set_bitflip(0.01)
noise.mov.loss = 0.001
results = device.simulate(qir, shots=1000, noise=noise, type="gpu")
Histogram(results, labels="kets", items="top-25")

## Hook up sliders to vary noise levels and see how results change

In [ ]:
from ipywidgets import interact, FloatSlider, HTML, VBox, SliderStyle
from IPython.display import display

histo = Histogram(None, labels="kets", items="top-25")

# Create sliders with styled readout values
depol = FloatSlider(
    value=0.01, min=0.0, max=0.1, step=0.005,
    style=SliderStyle(readout_color='gray')
)
depol_label = HTML(value='<span style="color: var(--vscode-foreground)">Depolarizing Rate</span>')

loss = FloatSlider(
    value=0.01, min=0.0, max=0.1, step=0.005,
    style=SliderStyle(readout_color='gray')
)
loss_label = HTML(value='<span style="color: var(--vscode-foreground)">Loss Rate</span>')

# Turn a list of strings into a dict of labels to counts
def to_buckets(result_list):
    buckets = {}
    for result in result_list:
        label = str(result)
        if label in buckets:
            buckets[label] += 1
        else:
            buckets[label] = 1
    return buckets

def run_with_noise(change = None):
    depolarizing_rate = depol.value
    loss_rate = loss.value

    noise = NoiseConfig()
    noise.cz.set_depolarizing(depolarizing_rate)
    noise.rz.set_depolarizing(depolarizing_rate)
    noise.h.set_depolarizing(depolarizing_rate)
    noise.mov.loss = loss_rate

    output = device.simulate(qir, shots=1024, noise=noise, type="gpu")
    histo.buckets = to_buckets(output)
    histo.shot_count = 1024

depol.observe(run_with_noise)
loss.observe(run_with_noise)

display(VBox([depol_label, depol]))
display(VBox([loss_label, loss]))
display(histo)

run_with_noise()

## Visualize the molecule we've been working with

In [ ]:
from pathlib import Path
from qdk.widgets import MoleculeViewer

# TODO: This data should come from calling qdk-chemistry Python APIs.
molecule_data = Path("benzene_diradical.structure.xyz").read_text()
cube_data = {
    "alpha_18": Path("MO_alpha_18.cube").read_text(),
    "alpha_19": Path("MO_alpha_19.cube").read_text(),
}

MoleculeViewer(molecule_data=molecule_data, cube_data=cube_data, isoval=0.03)